In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# ─── 0. LOAD PICKLE FILES ───────────────────────────────────────────────────────────
with open('steering_vector_for_.pkl', 'rb') as f:
    vector1_dict = pickle.load(f)   # dict[layer → [Tensor × K offsets]]

with open('steering_vectors_newbalancedtask3_meandiff_projectoutnuisance.pkl', 'rb') as f:
    vector2_dict = pickle.load(f)   # dict[layer → [Tensor × K offsets]]

with open('steering_vector_unnormalized_meandiff_normalized_nuisance_normalized_steering.pkl', 'rb') as f:
    vector3_dict = pickle.load(f)   # dict[layer → [Tensor × K offsets]]

# ─── CONVERT TO NUMPY ARRAYS ───────────────────────────────────────────────────────
vector1_list = [v.numpy() for vs in vector1_dict.values() for v in vs]
vector2_list = [v.numpy() for vs in vector2_dict.values() for v in vs]
vector3_list = [v.numpy() for vs in vector3_dict.values() for v in vs]

vector1_mat = np.vstack(vector1_list)  # (total_offsets, hidden_size)
vector2_mat = np.vstack(vector2_list)
vector3_mat = np.vstack(vector3_list)

# ─── GRAPH 1: NORM HISTOGRAM FOR VECTOR 1, VECTOR 2, VECTOR 3 ─────────────────────
norms_v1 = np.linalg.norm(vector1_mat, axis=1)
norms_v2 = np.linalg.norm(vector2_mat, axis=1)
norms_v3 = np.linalg.norm(vector3_mat, axis=1)

plt.figure(figsize=(8, 5))
plt.hist(norms_v1, bins=40, alpha=0.6, label="Vector 1")
plt.hist(norms_v2, bins=40, alpha=0.6, label="Vector 2")
plt.hist(norms_v3, bins=40, alpha=0.6, label="Vector 3")
plt.title("Graph 1: Norm Distribution of Vector 1 vs Vector 2 vs Vector 3")
plt.xlabel("‖v‖₂")
plt.ylabel("Count")
plt.legend()
plt.show()

# ─── GRAPH 2: PCA 2D SCATTER FOR VECTOR 1, VECTOR 2, VECTOR 3 ─────────────────────
all_mat = np.vstack([vector1_mat, vector2_mat, vector3_mat])
pca = PCA(n_components=2)
coords = pca.fit_transform(all_mat)

n1 = vector1_mat.shape[0]
n2 = vector2_mat.shape[0]
n3 = vector3_mat.shape[0]

coords_v1 = coords[:n1]
coords_v2 = coords[n1:n1+n2]
coords_v3 = coords[n1+n2:n1+n2+n3]

plt.figure(figsize=(8, 5))
plt.scatter(coords_v1[:, 0], coords_v1[:, 1], s=10, label="Vector 1", alpha=0.7)
plt.scatter(coords_v2[:, 0], coords_v2[:, 1], s=10, label="Vector 2", alpha=0.7)
plt.scatter(coords_v3[:, 0], coords_v3[:, 1], s=10, label="Vector 3", alpha=0.7)
plt.title("Graph 2: PCA 2D Scatter of Vector 1 vs Vector 2 vs Vector 3")
plt.xlabel("PC 1")
plt.ylabel("PC 2")
plt.legend()
plt.grid(True)
plt.show()

# ─── GRAPH 3: PER-LAYER MEAN NORM COMPARISON FOR VECTOR 1, VECTOR 2, VECTOR 3 ───────
num_layers = len(vector1_dict)            # e.g. 32
K = len(next(iter(vector1_dict.values())))  # e.g. 10

mean_norms_v1 = np.zeros(num_layers)
for i, layer in enumerate(sorted(vector1_dict.keys())):
    norms = [np.linalg.norm(v.numpy()) for v in vector1_dict[layer]]
    mean_norms_v1[i] = np.mean(norms)

mean_norms_v2 = np.zeros(num_layers)
for i, layer in enumerate(sorted(vector2_dict.keys())):
    norms = [np.linalg.norm(v.numpy()) for v in vector2_dict[layer]]
    mean_norms_v2[i] = np.mean(norms)

mean_norms_v3 = np.zeros(num_layers)
for i, layer in enumerate(sorted(vector3_dict.keys())):
    norms = [np.linalg.norm(v.numpy()) for v in vector3_dict[layer]]
    mean_norms_v3[i] = np.mean(norms)

layers = np.arange(num_layers)
plt.figure(figsize=(8, 5))
plt.plot(layers, mean_norms_v1, marker='o', label="Vector 1 Mean Norm")
plt.plot(layers, mean_norms_v2, marker='s', label="Vector 2 Mean Norm")
plt.plot(layers, mean_norms_v3, marker='^', label="Vector 3 Mean Norm")
plt.title("Graph 3: Per-Layer Mean Norms of Vector 1 vs Vector 2 vs Vector 3")
plt.xlabel("Layer index (0-based)")
plt.ylabel("Mean ‖v‖₂")
plt.legend()
plt.grid(True)
plt.show()
